# Podstawy analityki w Pythonie

Polecany notebook do nauki: https://infosecjupyterthon.com/workshops/day1/day1-3-Data-Analysis-with-Pandas%20Intro.html

In [ ]:
#!pip install pandas

In [1]:
import pandas as pd

**Pandas Series**

https://pandas.pydata.org/docs/reference/api/pandas.Series.html

In [ ]:
# standardowe index'y
data = ["Item 1", "Item 2", "Item 3"]
pd.Series(data, index=[1,2,3])

In [ ]:
# niestandardowe index'y
data = {"A": "Item 1", "B": "Item 2", "C": "Item 3"}
pd.Series(data)

**Pandas DataFrame**

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html

In [ ]:
data = {"Name": ["Item 1", "Item 2", "Item 3"], "Value": ["6.0", "3.2", "11.9"], "Count": [111, 720, 82]}
#pd.DataFrame(data)
#pd.DataFrame(data, index=["Item 1", "Item 2", "Item 3"])
df = pd.DataFrame(data).set_index('Name')
df

In [ ]:
# kolumny

df.columns

In [ ]:
#index'y wierszy

df.index

In [ ]:
df

In [ ]:
# odwołanie do konkretnej komórki - przykład

df.at["Item 1", "Value"]

In [ ]:
# odwołanie do konkretnego wiersza - przykład

df.loc["Item 2"]
#df.iloc[2]

In [ ]:
# odwołanie do kolumny/grupy kolumn - przykłady

df["Value"]
#df[["Value", "Count"]]

In [ ]:
# pojedyncza kolumna DataFrame to w rzeczywistości obiekt Pandas Series

type(df['Value'])

In [ ]:
type(df[["Value", "Count"]])

In [ ]:
# sprawdzenie typów danych w poszczególnych kolumnach

df.dtypes

In [ ]:
df

In [ ]:
# Dodanie nowej kolumny
df['UserName'] = ['SYSTEM', 'SYSTEM', 'MSTICAdmin']

In [ ]:
df

In [ ]:
# przykład odwołania do wierszy dla których mamy porządaną wartość w wybranej kolumnie

df.loc[df["UserName"] == "SYSTEM"]

In [ ]:
df[df["UserName"].str.endswith("Admin")]

In [ ]:
df[df["UserName"].str.contains("Admi")]

In [ ]:
df[df["UserName"].str.startswith("MS")]

In [ ]:
# odwołanie do konkretnych wierszy na podstawie wielu warunków

df[(df["Count"] <= 200) & (df["UserName"].isin(["SYSTEM", "Test"]))]

In [ ]:
# sortowanie wartości po wybranej kolumnie

df.sort_values(by = 'Count')

In [ ]:
# przykładowy wykres

df["Count"].plot(kind='bar') #kind - rodzaj wykresu, w tym wypadku słupkowy

Więcej podstaw w podlinkowanym na początku sekcji notebook'u. Pełna dokumentacja Pandas: https://pandas.pydata.org.

# Przykład: alert enrichment

## Krok 1 - import modułów

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import requests


## Krok 2 - definicja funkcji do pobierania danych lokalizacyjncyh

In [ ]:
def get_ip_location(ip):
    response = requests.get(f"http://ip-api.com/json/{ip}")
    data = response.json()
    if data['status'] == 'success':
        return {
            'kraj': data.get('country'),
            'miasto': data.get('city'),
            'dostawca_usług_internetowych': data.get('isp')
        }
    else:
        return None


## Krok 3 - Przykładowe dane

In [ ]:
data = {
    'alert_id': [1, 2, 3, 4, 5],
    'ip_address': ['8.8.8.8', '8.8.4.4', '212.77.100.83', '208.67.222.222', '194.29.133.3'],
    'severity': ['high', 'medium', 'low', 'medium', 'high'],
    'threat_type': ['malware', 'phishing', 'adware', 'malware', 'phishing'],
    'alert_time': pd.to_datetime(['2023-11-01 08:30', '2023-11-01 09:15', '2023-11-01 09:45', '2023-11-01 10:05', '2023-11-01 10:45'])
}
df = pd.DataFrame(data)


# Krok 4 - Funkcja do wzbogacania alertów (alert enrichment)

In [ ]:
def enrich_ip_addresses(df, ip_column):
    enriched_data = []
    for ip in df[ip_column]:
        location = get_ip_location(ip)
        if location:  # Sprawdzamy czy dostaliśmy lokalizację
            enriched_data.append(location)
        else:  # Jeśli nie, dodajemy puste dane dla zachowania struktury DataFrame
            enriched_data.append({'kraj': None, 'miasto': None, 'dostawca_usług_internetowych': None})
    return pd.concat([df, pd.DataFrame(enriched_data)], axis=1)


## Krok 5 - Wzbogacenie danych i wyświetlenie DataFrame

In [ ]:
enriched_df = enrich_ip_addresses(df, 'ip_address')
enriched_df


## Krok 6 - Prosta wizualizacja: histogram złoszeń wg. severity

In [ ]:
# Tworzymy histogram severities
enriched_df['severity'].value_counts().plot(kind='bar', color='skyblue')
plt.title('Histogram zgłoszeń według poziomu zagrożenia')
plt.xlabel('Poziom zagrożenia')
plt.ylabel('Liczba zgłoszeń')
plt.xticks(rotation=0)
plt.show()


## Krok 7 - WIzualizacja danych - liczba zgłoszeń na typ zagrożenia

In [ ]:
# Tworzymy wykres kołowy dla typów zagrożeń
enriched_df['threat_type'].value_counts().plot(kind='pie', autopct='%1.1f%%', startangle=90, colors=['tomato', 'gold', 'lightgreen'])
plt.title('Procentowy udział typów zagrożeń w zgłoszeniach')
plt.ylabel('')  # Usunięcie etykiety y, która jest domyślnie 'threat_type'
plt.show()


## Zadanie: Wyrysowanie wykresu częstotliwości występowania alertów w zależności od kraju

Stwórz wykres słupkowy, który pokaże rozkład liczby alertów w zależności od kraju pochodzenia. Dodaj tytuł wykresu oraz etykiety osi, aby wykres był czytelny.

**Wskazówka:** Wykorzystaj funkcję value_counts() na kolumnie zawierającej informacje o kraju, aby uzyskać liczbę wystąpień dla każdego kraju, a następnie użyj tej serii danych do stworzenia wykresu za pomocą metody plot() z Pandas lub biblioteki Matplotlib.

### Zadanie Krok 1 - Agregacja danych, aby uzyskać liczbę alertów dla każdego kraju

### Zadanie Krok 2 - Stwórz wykres słupkowy przedstawiający liczbę alertów dla poszczególnych krajów.